## 1. Khởi tạo & import

In [1]:
# thêm project root (thư mục chứa 'src/') vào sys.path
from pathlib import Path
import sys

ROOT = Path.cwd()
while not (ROOT / "src").is_dir() and ROOT.parent != ROOT:
    ROOT = ROOT.parent

sys.path.append(str(ROOT))

In [2]:
# 1) Imports thiết yếu
from typing import Tuple
import numpy as np

# Import hàm từ src
from src import ensure_data_dirs, kaggle_download_if_needed, load_airbnb_numpy, basic_checks
from src.data_processing import DATASET_NAME, FILENAME  # hằng số tên dataset/file

## 2. Đọc dữ liệu (CSV => NumPy)

In [3]:
# 2) Đảm bảo thư mục dữ liệu và file CSV
dirs = ensure_data_dirs(root="../data")
csv_path = dirs["raw"] / FILENAME
if not csv_path.exists():
    kaggle_download_if_needed(dataset=DATASET_NAME, filename=FILENAME, out_dir=str(dirs["raw"]))

# Đọc CSV thành cấu trúc NumPy thuần:
#   data["num"]: dict cột số -> np.ndarray
#   data["text"]: dict cột text -> np.ndarray(dtype=object)
#   data["header"]: list tên cột theo file
data = load_airbnb_numpy(csv_path)

# In nhanh vài kiểm tra cơ bản (để debug nếu cần)
report = basic_checks(data)
print({k: report[k] for k in ["n_rows","n_cols","out_of_range_avail","na_reviews_per_month","na_last_review"]})


{'n_rows': 48895, 'n_cols': 16, 'out_of_range_avail': 0, 'na_reviews_per_month': 10052, 'na_last_review': 10052}


## 3. Hàm helpers thuần NumPy 

In [4]:
def clip_outliers_percentile(x: np.ndarray, p_low=0.0, p_high=99.5) -> Tuple[np.ndarray, float, float]:
    """Clip theo phần trăm vị (để cắt đuôi cực trị). Trả về (x_clip, lo, hi)."""
    lo, hi = np.percentile(x, [p_low, p_high])
    return np.clip(x, lo, hi), float(lo), float(hi)

def one_hot(values: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """One-hot cho mảng object/string. Trả về (OH, uniques)."""
    uniq, inv = np.unique(values, return_inverse=True)
    oh = np.eye(uniq.size, dtype=np.float64)[inv]
    return oh, uniq

def nonempty_mask(arr_obj: np.ndarray) -> np.ndarray:
    """Mask True nếu phần tử không rỗng/NA (cho text)."""
    return np.array([str(v).strip() not in {"", "NA"} for v in arr_obj], dtype=bool)

def safe_log1p(x: np.ndarray) -> np.ndarray:
    """log1p an toàn cho dữ liệu không âm."""
    x = np.maximum(x, 0.0)
    return np.log1p(x)

## 4. Làm sạch dữ liệu

In [5]:
# 4) Lấy các cột cần thiết
num = data["num"]
txt = data["text"]

price = num["price"].astype(np.float64)  # mục tiêu y
minimum_nights = num["minimum_nights"].astype(np.float64)
number_of_reviews = num["number_of_reviews"].astype(np.float64)
host_listings = num["calculated_host_listings_count"].astype(np.float64)
availability_365 = num["availability_365"].astype(np.float64)
reviews_per_month = num["reviews_per_month"].astype(np.float64)  # có NaN

room_type = txt["room_type"]               # object array
neigh_group = txt["neighbourhood_group"]   # object array
last_review = txt["last_review"]           # object array

# 4.1 Chuẩn hóa phạm vi hợp lệ
# availability phải trong [0, 365] → clip cứng
availability_365 = np.clip(availability_365, 0, 365)

# 4.2 Xử lý thiếu (NaN): reviews_per_month
# Theo semantics dataset, thiếu thường ≈ "không có review" → điền 0
rpm_filled = reviews_per_month.copy()
rpm_filled[np.isnan(rpm_filled)] = 0.0

# 4.3 Ngoại lai (outliers) — cắt đuôi ở đặc trưng đầu vào (không đụng y lúc này)
min_nights_clip, mn_lo, mn_hi = clip_outliers_percentile(minimum_nights, p_low=0.0, p_high=99.0)
host_listings_clip, hl_lo, hl_hi = clip_outliers_percentile(host_listings, p_low=0.0, p_high=99.0)

# (Tùy chọn) Ghi chú ngưỡng để trace/debug
print(f"minimum_nights clip to [{mn_lo:.1f}, {mn_hi:.1f}] | "
      f"host_listings clip to [{hl_lo:.1f}, {hl_hi:.1f}]")

# 4.4 Cờ có/không có last_review (tránh parse ngày, giữ đơn giản)
has_last_review = nonempty_mask(last_review).astype(np.float64)


minimum_nights clip to [1.0, 45.0] | host_listings clip to [1.0, 232.0]


## 5. Tạo đặc trưng 

In [6]:
# 5) One-hot cho biến phân loại
oh_room, rt_uniques = one_hot(room_type)
oh_neigh, ng_uniques = one_hot(neigh_group)

# 5.1 Đặc trưng số biến đổi nhẹ để giảm lệch
num_reviews_log = safe_log1p(number_of_reviews)
min_nights_log = safe_log1p(min_nights_clip)
host_listings_log = safe_log1p(host_listings_clip)
days_avail_ratio = availability_365 / 365.0  # [0,1]
rpm_log1p = safe_log1p(rpm_filled)

# 5.2 Cờ tiện dụng
is_entire_home = (room_type == "Entire home/apt").astype(np.float64)
host_is_big = (host_listings_clip >= 3.0).astype(np.float64)  # ngưỡng đơn giản

# 5.3 Ghép ma trận đặc trưng X (float64)
X_blocks = [
    num_reviews_log.reshape(-1, 1),
    min_nights_log.reshape(-1, 1),
    host_listings_log.reshape(-1, 1),
    days_avail_ratio.reshape(-1, 1),
    rpm_log1p.reshape(-1, 1),
    is_entire_home.reshape(-1, 1),
    host_is_big.reshape(-1, 1),
    has_last_review.reshape(-1, 1),
    oh_room,   # one-hot room_type
    oh_neigh,  # one-hot neighbourhood_group
]
X = np.concatenate(X_blocks, axis=1)

# 5.4 Tạo y (giữ bản gốc) + biến đổi log để tiện thử nghiệm sau này
y = price.copy()
y_log1p = safe_log1p(y)

# 5.5 Tên cột (để debug dễ)
feature_names = [
    "log1p_num_reviews",
    "log1p_minimum_nights",
    "log1p_host_listings",
    "days_available_ratio",
    "log1p_reviews_per_month",
    "is_entire_home",
    "host_is_big_ge3",
    "has_last_review",
]
feature_names += [f"rt::{s}" for s in rt_uniques.tolist()]
feature_names += [f"ng::{s}" for s in ng_uniques.tolist()]

feature_names = np.array(feature_names, dtype=object)

print("X shape:", X.shape, "| y shape:", y.shape)


X shape: (48895, 16) | y shape: (48895,)


## 6. Lưu output .npz để dùng cho 03_modeling

In [7]:
# 6) Lưu vào data/processed với đầy đủ meta cần thiết
out_dir = dirs["processed"]
out_path = out_dir / "airbnb_2019_preprocessed.npz"

np.savez_compressed(
    out_path,
    X=X, 
    y=y, 
    y_log1p=y_log1p,
    feature_names=feature_names,
    room_type_uniques=rt_uniques.astype(object),
    neigh_group_uniques=ng_uniques.astype(object),
)

print("Saved ->", out_path)

Saved -> ../data/processed/airbnb_2019_preprocessed.npz


## 7) Sanity check 

In [8]:
# 7) Kiểm tra không có NaN/Inf, shape khớp
assert np.isfinite(X).all(), "X có NaN/Inf"
assert np.isfinite(y).all(), "y có NaN/Inf"
assert X.shape[0] == y.shape[0], "Mismatch số hàng giữa X và y"

# In vài thống kê để confirm
print("NaN trong rpm_filled:", int(np.sum(np.isnan(rpm_filled))))
print("Tỉ lệ Entire home/apt:", float(np.mean(is_entire_home)))
print("Số nhóm room_type:", rt_uniques.size, "| neighbourhood_group:", ng_uniques.size)


NaN trong rpm_filled: 0
Tỉ lệ Entire home/apt: 0.5196645873811229
Số nhóm room_type: 3 | neighbourhood_group: 5
